In [ ]:
import logging
from binance import Client, exceptions
import pandas as pd
import time
from datetime import datetime
import os
from dotenv import load_dotenv
from prometheus_client import start_http_server, Gauge, Counter, Histogram, Summary

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Cria Metricas do Prometheus

# Counter
total_trades_metric = Counter('trade_bot_total_trades', 'Total number of trades', ['currency'])
buy_attempts_metric = Counter('trade_bot_buy_attempts', 'Number of buy attempts', ['currency'])
successful_buys_metric = Counter('trade_bot_successful_buys', 'Number of successful buys', ['currency'])
successful_sells_metric = Counter('trade_bot_successful_sells', 'Number of successful sells', ['currency'])

# Gauge
total_loss_metric = Gauge('trade_bot_total_loss', 'Total loss accumulated', ['currency'])
trade_volume_metric = Gauge('trade_bot_trade_volume', 'Total volume of trades', ['currency'])
success_rate_metric = Gauge('trade_bot_success_rate', 'Success rate of trades', ['currency'])
total_profit_metric = Gauge('trade_bot_total_profit', 'Total profit accumulated', ['currency'])
current_price_metric = Gauge('trade_bot_current_price', 'Current price of the asset', ['currency'])
last_buy_price_metric = Gauge('trade_bot_last_buy_price', 'Last buy price', ['currency'])
last_sell_price_metric = Gauge('trade_bot_last_sell_price', 'Last sell price', ['currency'])
current_stoploss_metric = Gauge('trade_bot_current_stoploss', 'Current stoploss value', ['currency'])
current_stopgain_metric = Gauge('trade_bot_current_stopgain', 'Current stopgain value', ['currency'])
average_trade_duration_metric = Gauge('trade_bot_average_trade_duration', 'Average duration of trades', ['currency'])

# Summary
transaction_outcome_metric = Summary('trade_bot_transaction_outcome', 'Transaction outcomes (gain/loss)', ['currency'])

# Histogram
buy_duration_metric = Histogram('trade_bot_buy_duration_seconds', 'Duration of buy transactions in seconds', ['currency'])
sell_duration_metric = Histogram('trade_bot_sell_duration_seconds', 'Duration of sell transactions in seconds', ['currency'])
trade_latency_metric = Histogram('trade_bot_trade_latency_seconds', 'Latency of trade operations', ['currency'])

# Variáveis para monitorar o lucro, perda e duração total das trades
total_profit = 0
total_loss = 0
total_trade_duration = 0
successful_trades = 0
total_trades = 0

# Iniciar o servidor HTTP do Prometheus em uma thread separada
start_http_server(8000)

# Configure o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Obtenha as chaves da API de variáveis de ambiente
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')

if not api_key or not api_secret:
    logger.error("API Key ou API Secret não encontrada. Verifique o arquivo .env.")
    exit(1)

client = Client(api_key, api_secret)

symbol = "BTCUSDT"
quantity = 0.0011  # A quantidade de BTC transacionada
interval = '1h'  # O intervalo de tempo das velas
setup = "9.1"  # Identificador do setup de trading

def calculate_percentage(current_price, target_price):
    return (target_price - current_price) / current_price * 100

def read_state():
    if os.path.exists('data/bot_state.txt'):
        with open('data/bot_state.txt', 'r') as file:
            state = file.read().strip()
        if state == 'comprado':
            return True, read_trade_history()
    return False, pd.DataFrame()

def save_state(comprado, transaction_history=None):
    with open('data/bot_state.txt', 'w') as file:
        file.write('comprado' if comprado else 'não comprado')
    if transaction_history is not None:
        transaction_history.to_csv('data/trade_history.csv', index=False)

def read_trade_history():
    if os.path.exists('data/trade_history.csv'):
        df = pd.read_csv('data/trade_history.csv')
        if not df.empty:
            return df
    return pd.DataFrame()

def update_trade_history(df, sell_price):
    global total_profit, total_loss, total_trade_duration, successful_trades, total_trades
    df.at[df.index[-1], 'valor_venda'] = sell_price
    outcome = calculate_percentage(df.loc[df.index[-1], 'valor_compra'], sell_price)
    df.at[df.index[-1], 'outcome'] = outcome
    df.to_csv('data/trade_history.csv', index=False)
    transaction_outcome_metric.labels(symbol).observe(outcome)

    if outcome > 0:
        total_profit += outcome
        successful_trades += 1
    else:
        total_loss += outcome

    total_trades += 1
    total_profit_metric.labels(symbol).set(total_profit)
    total_loss_metric.labels(symbol).set(total_loss)
    success_rate = (successful_trades / total_trades) * 100 if total_trades > 0 else 0
    success_rate_metric.labels(symbol).set(success_rate)
    total_trades_metric.labels(symbol).inc()
    average_trade_duration = total_trade_duration / total_trades if total_trades > 0 else 0
    average_trade_duration_metric.labels(symbol).set(average_trade_duration)
    trade_volume_metric.labels(symbol).inc(quantity)

def update_metrics_on_buy(current_price, stoploss, stopgain):
    current_price_metric.labels(symbol).set(current_price)
    current_stoploss_metric.labels(symbol).set(stoploss)
    current_stopgain_metric.labels(symbol).set(stopgain)
    last_buy_price_metric.labels(symbol).set(current_price)
    buy_attempts_metric.labels(symbol).inc()
    successful_buys_metric.labels(symbol).inc()

def update_metrics_on_sell(ticker):
    last_sell_price_metric.labels(symbol).set(ticker)
    successful_sells_metric.labels(symbol).inc()

def main_loop():
    global total_trade_duration, total_trades

    while True:
        try:
            # Verifica o estado do bot
            comprado, trade_history = read_state()

            #logger.info("Bot v2 iniciado.")

            while not comprado:
                klines = client.get_klines(symbol=symbol, interval=interval, limit=50)
                data = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
                data['close'] = data['close'].astype(float)
                data['low'] = data['low'].astype(float)
                data['high'] = data['high'].astype(float)

                previous_ema = data['close'].ewm(span=9, adjust=False).mean().iloc[-2]
                pre_previous_ema = data['close'].ewm(span=9, adjust=False).mean().iloc[-3]
                current_price = data['close'].iloc[-1]
                previous_high = data['high'].iloc[-2]

                #logger.info("Loop de compra - Checando condições de compra.")

                if previous_ema > pre_previous_ema and current_price >= previous_high:
                    start_time = time.time()
                    order = client.order_market_buy(symbol=symbol, quantity=quantity)
                    buy_duration_metric.labels(symbol).observe(time.time() - start_time)
                    stoploss = data['low'].iloc[-2]
                    stopgain = previous_high * 1.02
                    potential_loss = calculate_percentage(current_price, stoploss)
                    potential_gain = calculate_percentage(current_price, stopgain)
                    logger.info(f"Compramos - Potencial de perda: {potential_loss:.2f}%, Potencial de ganho: {potential_gain:.2f}%")
                    comprado = True
                    new_row = pd.DataFrame({
                        'horario': [datetime.now()],
                        'moeda': [symbol],
                        'valor_compra': [current_price],
                        'valor_venda': [None],
                        'quantidade_moeda': [quantity],
                        'max_referencia': [previous_high],
                        'min_referencia': [data['low'].iloc[-2]],
                        'stoploss': [stoploss],
                        'stopgain': [stopgain],
                        'potential_loss': [potential_loss],
                        'potential_gain': [potential_gain],
                        'timeframe': [interval],
                        'setup': [setup],
                        'outcome': [None]
                    })
                    trade_history = pd.concat([trade_history, new_row], ignore_index=True)
                    save_state(comprado, trade_history)
                    update_metrics_on_buy(current_price, stoploss, stopgain)
                #else:
                    #logger.info("Aguardando condições de compra ideais.")
                time.sleep(1)

            while comprado:
                logger.info("Loop de venda - Checando condições de venda.")
                ticker = float(client.get_symbol_ticker(symbol=symbol)['price'])

                if not trade_history.empty:
                    stoploss = trade_history['stoploss'].iloc[-1]
                    stopgain = trade_history['stopgain'].iloc[-1]

                    if ticker <= stoploss:
                        start_time = time.time()
                        order = client.order_market_sell(symbol=symbol, quantity=quantity)
                        trade_duration = time.time() - start_time
                        sell_duration_metric.labels(symbol).observe(trade_duration)
                        total_trade_duration += trade_duration
                        comprado = False
                        logger.info("Fomos stopados :(")
                        update_trade_history(trade_history, ticker)
                        save_state(comprado, trade_history)
                        update_metrics_on_sell(ticker)
                    elif ticker >= stopgain:
                        start_time = time.time()
                        order = client.order_market_sell(symbol=symbol, quantity=quantity)
                        trade_duration = time.time() - start_time
                        sell_duration_metric.labels(symbol).observe(trade_duration)
                        total_trade_duration += trade_duration
                        comprado = False
                        logger.info("Alcançamos o stopgain! Vendendo...")
                        update_trade_history(trade_history, ticker)
                        save_state(comprado, trade_history)
                        update_metrics_on_sell(ticker)
                    #else:
                        #logger.info("Posição mantida")
                    time.sleep(1)
                else:
                    logger.info("Detalhes de transação não disponíveis. Aguarde a próxima oportunidade de compra.")
                    time.sleep(1)

        except exceptions.BinanceAPIException as e:
            logger.error(f"Erro na API Binance: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except exceptions.BinanceOrderException as e:
            logger.error(f"Erro ao criar ordem na Binance: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except Exception as e:
            logger.error(f"Erro inesperado: {e}")
            time.sleep(25)  # Espera antes de tentar novamente

if __name__ == "__main__":
    main_loop()